# 决策树

- error function:
  $$
  error = \frac{\text{#incorret num}}{\text{#example num}}
  $$


### 构建决策树

- 递归贪婪算法
  1. 从空树开始
  2. 选择一个特征开始分离数据
  3. 对于每个子树
    - 子树内为单一结果,则输出结果
    - 否则,重复第 2 步
  
1. 如何选择特征?
  - $\hat y = $ 分类中的大部分数据
  - 计算分类误差: $error = \frac{\hat y}{\text{#total num}}$
  - 对每个特征计算分类误差
  - 选择分类误差最小的特征
2. 何时终止?
  - 特征使用完
  - 所有数据都已分类
3. 处理数值
  - 先将特征值排序 {$v_1, v_2, ..., v_n$}
  - 遍历排序, 令 $t_i = \frac{v_i + v_{i+1}}{2}$, 计算分类误差 $h_j(x) >= t_i$
  - 找到分类误差最小值的 $t_*$
  
处理过拟合问题:
- 提前结束
  - 限制树的深度
  - 继续加深并不会降低分类误差时(短视而错过好的分支)
  - 子树下的数据量过少,停止递归 (e.g $N_\min = 10 ~ 100$)
- 剪枝
  - $L(T)$: 子节点数量
  - 修改成本函数: $C(T) = Error(T) + \lambda L(T)$ , 同时考量分类误差和树的复杂程度
  - 步骤:
    1. 选取待修剪的分支
    2. 判断去除分支后的总成本
    3. 如果总成本减小则修改该分支

部分数据丢失处理:
1. 放弃数据缺失的样本(缺失样本数据量小的时候适用)
2. 放弃整个缺失的特征
3. 猜测数据
  - 最多数的类别
  - 平均的数值
4. **将 Unknown 数据添加到特征分支上(推荐)**
  - 添加方法: 使分类误差最小
  - 优点: 可解决猜测数据带来的偏差

### 定义分类数据

编号| 色泽 | 根蒂 | 敲声 | 纹理 | 脐部 | 触感 | 好瓜
---|---|---|---|---|---|---|---
1  |  青绿 | 蜷缩 | 浊响 | 清晰 | 凹陷 | 硬滑 | 是 
2  |  乌黑 | 蜡缩 | 沉闷 | 清晰 | 凹陷 | 硬滑 | 是 
3  |  乌黑 | 蜡缩 | 浊响 | 清晰 | 凹陷 | 硬滑 | 是 
4  |  青绿 | 蜷缩 | 沉闷 | 清晰 | 凹陷 | 硬滑 | 是 
5  |  浅白 | 蜷缩 | 浊响 | 清晰 | 凹陷 | 硬滑 | 是 
6  |  青绿 | 稍蜷 | 浊响 | 清晰 | 稍凹 | 软粘 | 是 
7  |  乌黑 | 稍蜷 | 浊响 | 稍糊 | 稍凹 | 软粘 | 是 
8  |  乌黑 | 稍蜷 | 浊响 | 清晰 | 稍凹 | 硬滑 | 是 
9  |  乌黑 | 稍蜷 | 沉闷 | 稍糊 | 稍凹 | 硬滑 | 否 
10 |  青绿 | 硬挺 | 清脆 | 清晰 | 平坦 | 软粘 | 否 
11 |  洁白 | 硬挺 | 清脆 | 模糊 | 平坦 | 硬滑 | 否 
12 |  洁白 | 蜷缩 | 浊响 | 模糊 | 平坦 | 软粘 | 否 
13 |  青绿 | 稍蜷 | 浊响 | 稍糊 | 凹陷 | 硬滑 | 否 
14 |  浅白 | 稍蜷 | 沉闷 | 稍糊 | 凹陷 | 硬情 | 否 
15 |  乌黑 | 稍蜷 | 浊响 | 清晰 | 稍凹 | 软粘 | 否 
16 |  浅白 | 蜷缩 | 浊响 | 模糊 | 平坦 | 硬滑 | 否 
17 |  青绿 | 蜡缩 | 沉闷 | 稍糊 | 稍凹 | 硬滑 | 否 

In [1]:
import numpy as np

x = np.array([
    [3, 4, 3, 3, 3, 2], 
    [1, 1, 1, 3, 3, 2], 
    [1, 1, 3, 3, 3, 2], 
    [3, 4, 1, 3, 3, 2], 
    [2, 4, 3, 3, 3, 2], 
    [3, 2, 3, 3, 1, 1], 
    [1, 2, 3, 1, 1, 1], 
    [1, 2, 3, 3, 1, 2], 
    [1, 2, 1, 1, 1, 2], 
    [3, 3, 2, 3, 2, 1], 
    [2, 3, 2, 2, 2, 2], 
    [2, 4, 3, 2, 2, 1], 
    [3, 2, 3, 1, 3, 2], 
    [2, 2, 1, 1, 3, 2], 
    [1, 2, 3, 3, 1, 1], 
    [2, 4, 3, 2, 2, 2], 
    [3, 1, 1, 1, 1, 2]
], dtype=int)

y = np.array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int)

### 数据纯度计算

1. 信息熵
  - $p_k$: 第 k 类样本所占的比例($k = 1, 2, ..., |y|$)
  - 越低越纯
  $$
  Ent(D) = -\sum_{k=1}^{|y|}p_k\log_2p_k
  $$
2. 信息增益
  - 离散属性 a 有 V 个取值 $a^1, a^2, ..., a^V$
  - 越高越纯
  $$
  Gain(D) = Ent(D) - \sum_{v=1}^V\frac{|D^v|}{|D|}Ent(D^v)
  $$
2. 基尼指数: 数据集 D 中随机抽取 2 个样本不一致的概率
  - 越低越纯
  $$
  Gini(D) = \sum_{k=1}^{|y|}\sum_{k'\neq k}p_kp_{k'} = 1 - \sum_{k=1}^{|y|}p_k^2
  $$

In [2]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist / np.sum(hist)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

def gain(y, x):
    n_features = x.shape[1]
    gain = np.zeros(n_features) + entropy(y)
    m = len(x)
    for i in range(n_features):
        v = np.bincount(x[:, i])
        ent_v = np.sum([vv / m * entropy(y[x[:, i] == index]) for index, vv in enumerate(v) if vv > 0])
        gain[i] -= ent_v
    return gain

def gini(y):
    hist = np.bincount(y)
    N = np.sum(hist)
    return 1 - sum([(i / N) ** 2 for i in hist])

In [3]:
print(gain(y, x))

[0.10812517 0.14419446 0.14078143 0.3805919  0.28915878 0.00604649]


### 定义树结构

In [4]:
class Node:
    def __init__(self, left, right, rule):
        self.left = left
        self.right = right
        self.feature = rule[0]
        self.threshold = rule[1]


class Leaf:
    def __init__(self, value):
        self.value = value

### 具体算法

In [5]:
class DecisionTree:
    def __init__(
        self,
        classifier=True,
        max_depth=None,
        n_feats=None,
        criterion="entropy",
        seed=None,
    ):
        """
        A decision tree model for regression and classification problems.

        Parameters
        ----------
        classifier : bool
            Whether to treat target values as categorical (classifier =
            True) or continuous (classifier = False). Default is True.
        max_depth: int or None
            The depth at which to stop growing the tree. If None, grow the tree
            until all leaves are pure. Default is None.
        n_feats : int
            Specifies the number of features to sample on each split. If None,
            use all features on each split. Default is None.
        criterion : {'mse', 'entropy', 'gini'}
            The error criterion to use when calculating splits.
        seed : int or None
            Seed for the random number generator. Default is None.
        """
        if seed:
            np.random.seed(seed)

        self.depth = 0
        self.root = None

        self.n_feats = n_feats
        self.criterion = criterion
        self.classifier = classifier
        self.max_depth = max_depth if max_depth else np.inf


    def fit(self, X, Y):
        """
        Fit a binary decision tree to a dataset.

        Parameters
        ----------
        X : :py:class:`ndarray <numpy.ndarray>` of shape `(N, M)`
            The training data of `N` examples, each with `M` features
        Y : :py:class:`ndarray <numpy.ndarray>` of shape `(N,)`
            An array of integer class labels for each example in `X` if
            self.classifier = True, otherwise the set of target values for
            each example in `X`.
        """
        self.n_classes = max(Y) + 1 if self.classifier else None
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow(X, Y)

    def predict(self, X):
        """
        Use the trained decision tree to classify or predict the examples in `X`.

        Parameters
        ----------
        X : :py:class:`ndarray <numpy.ndarray>` of shape `(N, M)`
            The training data of `N` examples, each with `M` features

        Returns
        -------
        preds : :py:class:`ndarray <numpy.ndarray>` of shape `(N,)`
            The integer class labels predicted for each example in `X` if
            self.classifier = True, otherwise the predicted target values.
        """
        return np.array([self._traverse(x, self.root) for x in X])

    def predict_class_probs(self, X):
        """
        Use the trained decision tree to return the class probabilities for the
        examples in `X`.

        Parameters
        ----------
        X : :py:class:`ndarray <numpy.ndarray>` of shape `(N, M)`
            The training data of `N` examples, each with `M` features

        Returns
        -------
        preds : :py:class:`ndarray <numpy.ndarray>` of shape `(N, n_classes)`
            The class probabilities predicted for each example in `X`.
        """
        assert self.classifier, "`predict_class_probs` undefined for classifier = False"
        return np.array([self._traverse(x, self.root, prob=True) for x in X])

    def _grow(self, X, Y):
        # if all labels are the same, return a leaf
        if len(np.unique(Y)) == 1:
            if self.classifier:
                prob = np.zeros(self.n_classes)
                prob[Y[0]] = 1.0
            return Leaf(prob) if self.classifier else Leaf(Y[0])

        # if we have reached max_depth, return a leaf
        if self.depth >= self.max_depth:
            v = np.mean(Y, axis=0)
            if self.classifier:
                v = np.bincount(Y, minlength=self.n_classes) / len(Y)
            return Leaf(v)

        N, M = X.shape
        self.depth += 1
        feat_idxs = np.random.choice(M, self.n_feats, replace=False)

        # greedily select the best split according to `criterion`
        feat, thresh = self._segment(X, Y, feat_idxs)
        l = np.argwhere(X[:, feat] <= thresh).flatten()
        r = np.argwhere(X[:, feat] > thresh).flatten()

        # grow the children that result from the split
        left = self._grow(X[l, :], Y[l])
        right = self._grow(X[r, :], Y[r])
        return Node(left, right, (feat, thresh))

    def _segment(self, X, Y, feat_idxs):
        """
        Find the optimal split rule (feature index and split threshold) for the
        data according to `self.criterion`.
        """
        best_gain = -np.inf
        split_idx, split_thresh = None, None
        for i in feat_idxs:
            vals = X[:, i]
            levels = np.unique(vals)
            if len(levels) > 1:
                thresholds = (levels[:-1] + levels[1:]) / 2
            else:
                thresholds = levels
            gains = np.array([self._impurity_gain(Y, t, vals) for t in thresholds])

            if gains.max() > best_gain:
                split_idx = i
                best_gain = gains.max()
                split_thresh = thresholds[gains.argmax()]

        return split_idx, split_thresh

    def _impurity_gain(self, Y, split_thresh, feat_values):
        """
        Compute the impurity gain associated with a given split.

        IG(split) = loss(parent) - weighted_avg[loss(left_child), loss(right_child)]
        """
        if self.criterion == "entropy":
            loss = entropy
        else:
            loss = gini

        parent_loss = loss(Y)

        # generate split
        left = np.argwhere(feat_values <= split_thresh).flatten()
        right = np.argwhere(feat_values > split_thresh).flatten()

        if len(left) == 0 or len(right) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(Y)
        n_l, n_r = len(left), len(right)
        e_l, e_r = loss(Y[left]), loss(Y[right])
        child_loss = (n_l / n) * e_l + (n_r / n) * e_r

        # impurity gain is difference in loss before vs. after split
        ig = parent_loss - child_loss
        return ig

    def _traverse(self, X, node, prob=False):
        if isinstance(node, Leaf):
            if self.classifier:
                return node.value if prob else node.value.argmax()
            return node.value
        if X[node.feature] <= node.threshold:
            return self._traverse(X, node.left, prob)
        return self._traverse(X, node.right, prob)

### 训练

In [6]:
dt = DecisionTree()
dt.fit(x, y)

### 测试

In [7]:
print(dt.predict(np.array([
    [2, 4, 3, 3, 3, 2],
    [1, 2, 3, 3, 1, 1],
])))

[1 0]
